Consider a stock $S(t)$ as the solution to the following SDE:
$$dS(t) = \mu(S(t),t) dt + \sigma(S(t),t) dW(t), \quad 0 \le t \le T$$

The Euler discretization is given by,
$$s_{i+1} = s_i + \mu (s_i,t_i) \Delta t + \sigma(s_i,t_i) \Delta W_i$$

where,
$$\Delta W_i = W_{i+1} - W_i = Z \sqrt{(\Delta t)}, \quad Z \sim N(0,1)$$

The Milstein discretization improves the convergence speed for a one-dimensional SDE problem and is given by,

$$s_{i+1} = s_i + \mu (s_i,t_i) \Delta t + \sigma(s_i,t_i) \Delta W_i + \frac{1}{2} \sigma^2 s_i ( (\Delta W_i )^2 - \Delta t)$$

The procedure implemented below follows five steps.

1. Discretize the time dimension to $[0,T]$ i.e. $0 =t_0 < t_1 < \dots < t_m = T$

2. Generate asset values $s_{i,j}$ using discretization schemes such as Euler/ Milstein. Here, $i$ is the time step and $j$ is the iteration number in Monte Carlo simulation

3. Compute $N$ payoff values $H_j = H(T,s_{m,j})$

4. Compute average: $\frac{1}{N} \sum_{j=1}^N H_j$

5. Evaluate option value $$exp\left(-r (T-t_0)\right) \frac{1}{N} \sum_{j=1}^N H_j$$

6. Determine standard error $\epsilon_N = \frac{v_N}{\sqrt{N}}$, where $$v_N^2 = \frac{1}{N-1} \sum_{j=1}^N \left[ H(T,s_{m,j}) - \frac{1}{N} \sum_{j=1}^N H(T,s) \right]^2$$

In [3]:
import numpy as np
import scipy.stats as st
import enum 

# set i= imaginary number
i   = complex(0.0,1.0)

# This class defines puts and calls
class OptionType(enum.Enum):
    CALL = 1.0
    PUT = -1.0
    
# Black-Scholes Call option price
def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    
    K = np.array(K).reshape([len(K),1])
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) 
    * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if CP == OptionType.CALL:
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif CP == OptionType.PUT:
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

def GeneratePathsGBMEuler(NoOfPaths,NoOfSteps,T,r,sigma,S_0):    
    Z = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    W = np.zeros([NoOfPaths, NoOfSteps+1])
   
    # Euler Approximation
    S1 = np.zeros([NoOfPaths, NoOfSteps+1])
    S1[:,0] =S_0
    
    time = np.zeros([NoOfSteps+1])
        
    dt = T / float(NoOfSteps)
    for i in range(0,NoOfSteps):
        # making sure that samples from normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:,i] = (Z[:,i] - np.mean(Z[:,i])) / np.std(Z[:,i])
        W[:,i+1] = W[:,i] + np.power(dt, 0.5)*Z[:,i]
        
        S1[:,i+1] = S1[:,i] + r * S1[:,i]* dt + sigma * S1[:,i] * (W[:,i+1] - W[:,i])
        time[i+1] = time[i] +dt
        
    # Retun S1 and S2
    paths = {"time":time,"S":S1}
    return paths

def GeneratePathsGBMMilstein(NoOfPaths,NoOfSteps,T,r,sigma,S_0):    
    Z = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    W = np.zeros([NoOfPaths, NoOfSteps+1])
   
    # Milstein Approximation
    S1 = np.zeros([NoOfPaths, NoOfSteps+1])
    S1[:,0] =S_0
       
    time = np.zeros([NoOfSteps+1])
        
    dt = T / float(NoOfSteps)
    for i in range(0,NoOfSteps):
        # making sure that samples from normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:,i] = (Z[:,i] - np.mean(Z[:,i])) / np.std(Z[:,i])
        W[:,i+1] = W[:,i] + np.power(dt, 0.5)*Z[:,i]
        
        S1[:,i+1] = S1[:,i] + r * S1[:,i]* dt + sigma * S1[:,i] * (W[:,i+1] - W[:,i]) \
                    + 0.5 * sigma **2.0 * S1[:,i] * (np.power((W[:,i+1] - W[:,i]),2) - dt)
        time[i+1] = time[i] +dt
        
    # Retun S1 and S2
    paths = {"time":time,"S":S1}
    return paths

def EUOptionPriceFromMCPaths(CP,S,K,T,r):
    # S is a vector of Monte Carlo samples at T
    if CP == OptionType.CALL:
        return np.exp(-r*T)*np.mean(np.maximum(S-K,0.0))
    elif CP == OptionType.PUT:
        return np.exp(-r*T)*np.mean(np.maximum(K-S,0.0))

def mainCalculation():
    CP= OptionType.CALL
    T = 1
    r = 0.06
    sigma = 0.3
    S_0 = 5
    K = [S_0]
    NoOfSteps =1000
    
    # Simulated paths
    NoOfPathsV = [100,1000,5000,10000]
    
    # Call price
    exactPrice = BS_Call_Option_Price(CP,S_0,K,sigma,T,r)[0]
    print("European Option Price")
    print("Exact option price = {0}".format(exactPrice))
    for NoOfPathsTemp in NoOfPathsV:
        np.random.seed(1)
        PathsEuler    = GeneratePathsGBMEuler(NoOfPathsTemp,NoOfSteps,T,r,sigma,S_0)
        np.random.seed(1)
        PathsMilstein = GeneratePathsGBMMilstein(NoOfPathsTemp,NoOfSteps,T,r,sigma,S_0)
        S_Euler = PathsEuler["S"]
        S_Milstein = PathsMilstein["S"]
        priceEuler = EUOptionPriceFromMCPaths(CP,S_Euler[:,-1],K,T,r)
        priceMilstein = EUOptionPriceFromMCPaths(CP,S_Milstein[:,-1],K,T,r)
        print("For N = {0} Euler scheme yields option price = {1} and Milstein {2}"\
              .format(NoOfPathsTemp,priceEuler,priceMilstein))
        print("For N = {0} Euler error = {1} and Milstein  error {2}"\
              .format(NoOfPathsTemp,priceEuler-exactPrice,priceMilstein-exactPrice))
    
mainCalculation()

European Option Price
Exact option price = [0.73585362]
For N = 100 Euler scheme yields option price = 0.7285547612550968 and Milstein 0.728444051871542
For N = 100 Euler error = [-0.00729886] and Milstein  error [-0.00740957]
For N = 1000 Euler scheme yields option price = 0.7200580888738554 and Milstein 0.7202098357606097
For N = 1000 Euler error = [-0.01579553] and Milstein  error [-0.01564379]
For N = 5000 Euler scheme yields option price = 0.7481360160804368 and Milstein 0.7481404588408821
For N = 5000 Euler error = [0.0122824] and Milstein  error [0.01228684]
For N = 10000 Euler scheme yields option price = 0.7387184512149807 and Milstein 0.7386727436156081
For N = 10000 Euler error = [0.00286483] and Milstein  error [0.00281912]
